In [ ]:
from data_processing_helper import * 

In [ ]:
# Load a subject's data 
filename = "B0203T.csv"
eeg_filename = "./data/BCI_Competition/train/" + filename
event_filename = "./data/BCI_Competition/y/" + filename

eeg_chans = ["C3", "Cz", "C4"] # 10-20 system 
eog_chans = ["EOG:ch01", "EOG:ch02", "EOG:ch03"] 
all_chans = eeg_chans + eog_chans
event_types = {1:"left", 2:"right"}

# Load the raw csvs into dataframes
eeg_df = pd.read_csv(eeg_filename)
event_df = pd.read_csv(event_filename, names=["EventType"])

In [ ]:
eeg_df.head(2)

In [ ]:
event_df.head(2)

In [ ]:
# Extract data from raw dataframes for constructing trial-by-trial dataframe
trial_duration_ms = 4000
epochs = []
epoch_times = []
labels = []
start_df = eeg_df[eeg_df['FeedBackEvent'] == 1]
for i, event_type in enumerate(event_df["EventType"].values): 
    labels.append(event_type)
    start_time = start_df.iloc[i]["time"]
    end_time = int(start_time + trial_duration_ms)
    epoch_times.append((start_time, end_time))
    sub_df = eeg_df[(eeg_df['time'] > start_time) & (eeg_df['time'] <= end_time)]
    eeg_dat = []
    for ch in all_chans: 
        eeg_dat.append(sub_df[ch].values)
    epochs.append(np.array(eeg_dat))
    
# Create dataframe from the data extracted previously
eeg_epoch_df = getDF(epochs, labels, epoch_times, all_chans)

In [ ]:
# Preview dataframe of trials
eeg_epoch_df.head(2)

In [ ]:
# Visualize EEG and PSD for one trial
trial_num = 2

plt.figure(figsize=(15,5))
for ch in eeg_chans: 
    plt.plot(eeg_epoch_df[ch][trial_num], label=ch)
plt.ylabel("Voltage (uV)")
plt.xlabel("timepoints @ 250Hz")
plt.title("EEG of one motor imagery trial")
plt.legend() 
plt.show()

plt.figure(figsize=(15,5))
for ch in eog_chans: 
    plt.plot(eeg_epoch_df[ch][trial_num], label=ch)
plt.ylabel("Voltage (uV)")
plt.xlabel("timepoints @ 250Hz")
plt.title("EOG of one motor imagery trial")
plt.legend() 
plt.show()

plt.figure(figsize=(15,5))
for ch in eeg_chans: 
    plotPSD_fromEEG(eeg_epoch_df.iloc[trial_num][ch], pre_cut_off_freq=2, post_cut_off_freq=30, label=ch)
plt.title("PSD of one motor imagery trial")
plt.legend()
plt.show()


In [ ]:
# Get PSD averages for each channel for each event type (1=left or 2=right)
psd_averages_by_type = {}

for event_type in range(1, 3): 
    psds_only_one_type={}
    freqs_only_one_type={}
    for i, row in eeg_epoch_df[eeg_epoch_df["event_type"] == event_type].iterrows(): 
        for ch in eeg_chans: 
            if ch not in psds_only_one_type: 
                psds_only_one_type[ch] = list()
                freqs_only_one_type[ch] = list()
            f, p = getMeanFreqPSD(row[ch])
            psds_only_one_type[ch].append(p)
            freqs_only_one_type[ch].append(f)
    avg_psds_one_type = {}
    for ch in eeg_chans:
        psds_only_one_type[ch] = np.array(psds_only_one_type[ch])
        avg_psds_one_type[ch] = np.mean(psds_only_one_type[ch], axis=0)
    psd_averages_by_type[event_type] = dict(avg_psds_one_type)

In [ ]:
# View Average PSDs
for event_type in range(1, 3): 
    for ch in eeg_chans[:]: 
        plotPSD(freqs_only_one_type[eeg_chans[0]][0], psd_averages_by_type[event_type][ch],pre_cut_off_freq=2, post_cut_off_freq=30, label=ch)

    plt.legend()
    plt.title("event type: " + event_types[event_type])
    plt.show()